In [2]:
pip install mysql-connector-python pandas gspread oauth2client sqlalchemy rich openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sqlalchemy import create_engine, text


# url_o = "qa-delivery.cluster-ctxzyzjrixle.us-east-2.rds.amazonaws.com"

url = "qa-delivery-green-1uaykx.cluster-ctxzyzjrixle.us-east-2.rds.amazonaws.com"
# Database credentials
DATABASE_URL = f"mysql+mysqlconnector://qa-general:5twnhWXLmLvXzwkL@{url}:3306/delivery"
engine = create_engine(DATABASE_URL)


def call_query(sql_query) -> pd.DataFrame:
    try:
        # Execute query using SQLAlchemy engine
        with engine.connect() as connection:
            df = pd.read_sql(sql_query, connection)
            return df
    except Exception as err:
        print(f"Error: {err}")
        return None


def execute_update(sql_query) -> bool:
    try:
        # Ejecutar la consulta de actualización usando el motor de SQLAlchemy
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            # Verificar si la consulta afectó alguna fila
            if result.rowcount > 0:
                print("Actualización exitosa.")
                return True
            else:
                print("No se afectaron filas.")
                return False
    except Exception as err:
        print(f"Error: {err}")
        return False


def execute_query_from_file(filepath):
    try:
        with open(filepath, "r") as file:
            sql_query = file.read()
        return sql_query
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


def get_all_views_schema():
    """
    Retrieves the schema of all views in the database and saves them as SQL files.
    """
    try:
        # Query to fetch all view names
        query_views = f"""
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.VIEWS 
        WHERE TABLE_SCHEMA = 'delivery';
        """
        views = call_query(query_views)

        if views is not None and not views.empty:
            for view_name in views["TABLE_NAME"]:
                # Query to get the view schema
                query_view_schema = f"SHOW CREATE VIEW `{view_name}`;"
                view_schema = call_query(query_view_schema)

                if view_schema is not None and not view_schema.empty:
                    # Save the schema to a file
                    create_statement = view_schema.iloc[0]["Create View"]

                    # Save original view definition
                    with open(f"views/{view_name}.sql", "w", encoding="utf-8") as file:
                        file.write(create_statement)

                    # Extract and save just the SELECT portion
                    select_statement = create_statement[
                        create_statement.lower().find("select") :
                    ]
                    with open(
                        f"views_test/{view_name}.sql", "w", encoding="utf-8"
                    ) as file:
                        file.write(select_statement)
                    print(f"View '{view_name}' schema saved to views/{view_name}.sql")
                    print(
                        f"View '{view_name}' select saved to views_test/{view_name}.sql"
                    )
        else:
            print("No views found in the database.")
    except Exception as e:
        print(f"Error retrieving views: {e}")


def export_excel(data_frame, excel_filename="df"):
    data_frame.to_excel(excel_filename + ".xlsx", index=False)
    print(f"Datos exportados a {excel_filename}")

In [9]:
bf_source = """
select distinct X.IdPedido AS IdPedido,
    X.Codigo AS Codigo,
    X.Correlativo AS Correlativo,
    X.IdComercio AS IdComercio,
    X.Comercio AS Comercio,
    X.IdSucursal AS IdSucursal,
    X.Sucursal AS Sucursal,
    X.DireccionSucursal AS DireccionSucursal,
    X.TelefonoSucursal AS TelefonoSucursal,
    X.FechaCreacion AS FechaCreacion,
    X.FechaEstimadaEntregaMin AS FechaEstimadaEntregaMin,
    X.FechaEstimadaEntregaMax AS FechaEstimadaEntregaMax,
    X.EstadoPagoId AS EstadoPagoId,
    X.EstadoPago AS EstadoPago,
    X.EstadoPedidoId AS EstadoPedidoId,
    X.EstadoPedido AS EstadoPedido,
    X.FechaPago AS FechaPago,
    X.TipoEntregaId AS TipoEntregaId,
    X.TipoEntrega AS TipoEntrega,
    X.CodigoTipoPedido AS CodigoTipoPedido,
    X.TipoPedido AS TipoPedido,
    X.PrefijoTipoPedido AS PrefijoTipoPedido,
    X.EsPedidoHijo AS EsPedidoHijo,
    X.Canal AS Canal,
    X.Subtotal AS Subtotal,
    X.DeliveryCharge AS DeliveryCharge,
    X.DeliveryChargeIva AS DeliveryChargeIva,
    X.DeliveryAgilCharge AS DeliveryAgilCharge,
    X.DeliveryAgilChargeIva AS DeliveryAgilChargeIva,
    X.IvaPais AS IvaPais,
    X.RetencionRider AS RetencionRider,
    X.Neto AS Neto,
    X.Iva AS Iva,
    X.Bruto AS Bruto,
    X.GastoEnvio AS GastoEnvio,
    X.Propina AS Propina,
    X.PropinaAgilDelivery AS PropinaAgilDelivery,
    X.PropinaAgilRetiro AS PropinaAgilRetiro,
    X.PropinaComercioRetiro AS PropinaComercioRetiro,
    X.CodigoCupon AS CodigoCupon,
    X.DsctoCuponSubtotal AS DsctoCuponSubtotal,
    X.DsctoCuponGastoEnvio AS DsctoCuponGastoEnvio,
    X.DsctoPuntos AS DsctoPuntos,
    X.TotalBoleta AS TotalBoleta,
    X.IdMedioPago AS IdMedioPago,
    X.MedioPago AS MedioPago,
    X.MedioPagoIva AS MedioPagoIva,
    X.Moneda AS Moneda,
    X.TipoComision AS TipoComision,
    X.ComisionPagoPorcentaje AS ComisionPagoPorcentaje,
    X.ComisionPagoNeto AS ComisionPagoNeto,
    X.ComisionPagoIva AS ComisionPagoIva,
    X.ComisionPagoBruto AS ComisionPagoBruto,
    X.ComisionVentaPorcentaje AS ComisionVentaPorcentaje,
    X.ComisionVentaNeto AS ComisionVentaNeto,
    X.ComisionVentaIva AS ComisionVentaIva,
    X.ComisionVentaBruto AS ComisionVentaBruto,
    X.ComisionDeliveryPorcentaje AS ComisionDeliveryPorcentaje,
    X.ComisionDeliveryNeto AS ComisionDeliveryNeto,
    X.ComisionDeliveryIva AS ComisionDeliveryIva,
    X.ComisionDeliveryBruto AS ComisionDeliveryBruto,
    X.DiaDepositoComercio AS DiaDepositoComercio,
    X.CantidadDiasPeriodo AS CantidadDiasPeriodo,
    X.IdSucursalDeposito AS IdSucursalDeposito,
    X.PeriodoFacturacionInicioComercio AS PeriodoFacturacionInicioComercio,
    X.PeriodoFacturacionFinComercio AS PeriodoFacturacionFinComercio,
    X.DiaDepositoRider AS DiaDepositoRider,
    X.PeriodoFacturacionInicioRider AS PeriodoFacturacionInicioRider,
    X.PeriodoFacturacionFinRider AS PeriodoFacturacionFinRider,
    X.IdRider AS IdRider,
    X.IdPublicoRider AS IdPublicoRider,
    X.DniRider AS DniRider,
    X.NombreRider AS NombreRider,
    X.ApellidoRider AS ApellidoRider,
    X.CorreoRider AS CorreoRider,
    X.TarifaBaseRider AS TarifaBaseRider,
    X.IngresoExtraRider AS IngresoExtraRider,
    X.IngresoNetoRider AS IngresoNetoRider,
    X.RetencionHonorarioRider AS RetencionHonorarioRider,
    X.IngresoBrutoRider AS IngresoBrutoRider,
    X.IngresoViajeRider AS IngresoViajeRider,
    X.PropinaRider AS PropinaRider,
    X.TotalOfertaRider AS TotalOfertaRider,
    X.DepositoRider AS DepositoRider,
    X.IdCliente AS IdCliente,
    X.TipoPago AS TipoPago,
    X.Id_TipoVehiculo AS Id_TipoVehiculo,
    X.Descripcion AS Descripcion,
    X.CuotaServicioAgilNeto AS CuotaServicioAgilNeto,
    X.CuotaServicioAgilIVA AS CuotaServicioAgilIVA,
    X.CuotaServicioAgilBruto AS CuotaServicioAgilBruto,
    X.CuotaPasarelaNeto AS CuotaPasarelaNeto,
    X.CuotaPasarelaIVA AS CuotaPasarelaIVA,
    X.CuotaPasarelaBruto AS CuotaPasarelaBruto,
    X.FeePay AS FeePay,
    (
        (
            (
                (
                    (
                        (
                            (
                                (
                                    (
                                        (
                                            ifnull(X.Propina, 0) + ifnull(X.PropinaAgilDelivery, 0)
                                        ) + ifnull(X.PropinaAgilRetiro, 0)
                                    ) + ifnull(X.PropinaComercioRetiro, 0)
                                ) + ifnull(X.Bruto, 0)
                            ) + ifnull(X.ComisionPagoBruto, 0)
                        ) + ifnull(X.ComisionVentaBruto, 0)
                    ) + ifnull(X.ComisionDeliveryBruto, 0)
                ) + ifnull(X.GastoEnvio, 0)
            ) + ifnull(X.CuotaServicioAgilBruto, 0)
        ) + ifnull(X.CuotaPasarelaBruto, 0)
    ) AS TotalDescuentos,
    CASE
        WHEN IdMedioPago = 14
        or IdMedioPago = 15
        or IdMedioPago = 16 THEN -- Cálculo Post Pago
        (
            0 - ifnull(X.ComisionVentaBruto, 0) - ifnull(X.ComisionDeliveryBruto, 0) - ifnull(X.CuotaServicioAgilBruto, 0)
        )
        ELSE -- Cálculo Tradicional
        (
            ifnull(X.Subtotal, 0) - (
                ifnull(X.ComisionPagoBruto, 0) + ifnull(X.ComisionVentaBruto, 0) + ifnull(X.ComisionDeliveryBruto, 0) + ifnull(X.CuotaPasarelaBruto, 0) + ifnull(X.DsctoCuponSubtotal, 0) + ifnull(X.DsctoCuponGastoEnvio, 0) + ifnull(X.DsctoPuntos, 0)
            ) + ifnull(X.PropinaComercioRetiro, 0)
        )
    END AS DepositoComercio,
    (
        ifnull(X.ComisionVentaBruto, 0) + ifnull(X.ComisionDeliveryBruto, 0)
    ) AS TotalComisionesAgil,
    (
        (
            (
                ifnull(X.ComisionVentaBruto, 0) + ifnull(X.ComisionDeliveryBruto, 0)
            ) + ifnull(X.DeliveryAgilCharge, 0)
        ) + ifnull(X.DeliveryAgilChargeIva, 0)
    ) AS BrutoAgil,
    (
        (
            ifnull(X.ComisionVentaNeto, 0) + ifnull(X.ComisionDeliveryNeto, 0)
        ) + ifnull(X.DeliveryAgilCharge, 0)
    ) AS NetoAgil,
    (
        ifnull(X.DeliveryChargeIva, 0) + ifnull(X.DeliveryAgilChargeIva, 0)
    ) AS TotalDeliveryIva
from (
        select P.IdPedido AS IdPedido,
            P.Codigo AS Codigo,
            P.Corelativo AS Correlativo,
            C.IdComercio AS IdComercio,
            C.Nombre AS Comercio,
            S.IdSucursal AS IdSucursal,
            S.Titulo AS Sucursal,
            S.Direccion AS DireccionSucursal,
            S.Telefono AS TelefonoSucursal,
            P.FechaCreacion AS FechaCreacion,
            PE.FechaEstimadaEntrega AS FechaEstimadaEntregaMin,
            PE.FechaEstimadaEntregaMax AS FechaEstimadaEntregaMax,
            PPE.Id AS EstadoPagoId,
            PPE.Nombre AS EstadoPago,
            PES.Id AS EstadoPedidoId,
            PES.Nombre AS EstadoPedido,
            date_format(PPR.fecha, '%Y-%m-%d %H:%i:00') AS FechaPago,
            PTE.Id AS TipoEntregaId,
            PTE.Tipo AS TipoEntrega,
            TPD.Codigo AS CodigoTipoPedido,
            TPD.Nombre AS TipoPedido,
            TPD.Prefijo AS PrefijoTipoPedido,
            if(
                isnull(P.IdPedidoBase),
                'NO',
                'SI'
            ) AS EsPedidoHijo,
            'WEB' AS Canal,
            P.SubTotal AS Subtotal,
            P.DeliveryCharge AS DeliveryCharge,
            P.IvaDeliveryCharge AS DeliveryChargeIva,
            P.DeliveryAgilCharge AS DeliveryAgilCharge,
            P.IvaDeliveryAgilCharge AS DeliveryAgilChargeIva,
            ifnull(PP.MedioPagoIVA, N.IVA) AS IvaPais,
            N.IVA_Reducido AS RetencionRider,
            (P.DeliveryCharge + P.DeliveryAgilCharge) AS Neto,
            P.TotalIvaDelivery AS Iva,
            (
                (P.DeliveryCharge + P.DeliveryAgilCharge) + P.TotalIvaDelivery
            ) AS Bruto,
            ifnull(P.GastosEnvio, 0) AS GastoEnvio,
            P.Propina AS Propina,
            if(
                (PTE.Tipo = 'Delivery'),
                P.PropinaAgil,
                0
            ) AS PropinaAgilDelivery,
            if(
                (PTE.Tipo = 'Retiro en Tienda'),
                ifnull(
                    (
                        P.PropinaAgil - (
                            P.PropinaAgil * (ifnull(P.ComisionPropinaRetiro, 0) / 100)
                        )
                    ),
                    0
                ),
                0
            ) AS PropinaAgilRetiro,
            if(
                (PTE.Tipo = 'Retiro en Tienda'),
                ifnull(
                    (
                        P.PropinaAgil * (ifnull(P.ComisionPropinaRetiro, 0) / 100)
                    ),
                    0
                ),
                0
            ) AS PropinaComercioRetiro,
            SC.Codigo AS CodigoCupon,
            P.DsctoCuponSubtotal AS DsctoCuponSubtotal,
            P.DsctoCuponGastoEnvio AS DsctoCuponGastoEnvio,
            P.DsctoPuntos AS DsctoPuntos,
            P.Total_OC AS TotalBoleta,
            MP.id AS IdMedioPago,
            MP.medio_pago AS MedioPago,
            PP.MedioPagoIVA AS MedioPagoIva,
            PP.Moneda AS Moneda,
            (
                case
                    when (
                        (TPD.Codigo = 'PEDNORMAL')
                        and (PTE.Tipo = 'Delivery')
                    ) then 'NORMAL_WEB_DELIVERY'
                    when (
                        (TPD.Codigo = 'PEDNORMAL')
                        and (PTE.Tipo = 'Retiro en Tienda')
                    ) then 'NORMAL_WEB_RETIRO'
                    when (
                        (TPD.Codigo = 'PEDNORMAL')
                        and (PTE.Tipo = 'Delivery')
                    ) then 'NORMAL_APP_DELIVERY'
                    when (
                        (TPD.Codigo = 'PEDNORMAL')
                        and (PTE.Tipo = 'Retiro en Tienda')
                    ) then 'NORMAL_APP_RETIRO'
                    when (
                        (TPD.Codigo = 'PEDNORMAL')
                        and (PTE.Tipo = 'Delivery')
                    ) then 'NORMAL_APP_COMERCIO_DELIVERY'
                    when (
                        (
                            if(
                                isnull(P.IdPedidoBase),
                                'NO',
                                'SI'
                            ) = 'SI'
                        )
                        and (PTE.Tipo = 'Delivery')
                    ) then 'HIJO_POS_DELIVERY'
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PTE.Tipo = 'Delivery')
                    ) then 'ESPECIAL_POS_DELIVERY'
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PTE.Tipo = 'Retiro en Tienda')
                    ) then 'ESPECIAL_POS_RETIRO'
                    when (
                        (TPD.Codigo = 'PEDAGILFAVOR')
                        and (PTE.Tipo = 'Delivery')
                    ) then 'FAVOR_POS_DELIVERY'
                    when (
                        (TPD.Codigo = 'PEDMESA')
                        and (PTE.Tipo = 'Mesa')
                    ) then 'MESA'
                end
            ) AS TipoComision,
            if(
                (TPD.Codigo = 'PEDAGILFAVOR'),
                0,
                PP.MedioPagoFee
            ) AS ComisionPagoPorcentaje,
            if(
                (TPD.Codigo = 'PEDAGILFAVOR'),
                0,
                PPR.comision
            ) AS ComisionPagoNeto,
            if(
                (TPD.Codigo = 'PEDAGILFAVOR'),
                0,
                PPR.impuestos
            ) AS ComisionPagoIva,
            if(
                (TPD.Codigo = 'PEDAGILFAVOR'),
                0,
                (PPR.comision + PPR.impuestos)
            ) AS ComisionPagoBruto,
            (
                case
                    when (
                        if(
                            isnull(P.IdPedidoBase),
                            'NO',
                            'SI'
                        ) = 'SI'
                    ) then 0
                    when (TPD.Codigo = 'PEDAGILFAVOR') then 0
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PPE.Nombre = 'Activo')
                    ) then CS1.Valor
                    when (TPD.Codigo in ('PEDNORMAL', 'PEDMESA')) then CS1.Valor
                end
            ) AS ComisionVentaPorcentaje,
            (
                case
                    when (
                        if(
                            isnull(P.IdPedidoBase),
                            'NO',
                            'SI'
                        ) = 'SI'
                    ) then 0
                    when (TPD.Codigo = 'PEDAGILFAVOR') then 0
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PPE.Nombre = 'Activo')
                    ) then (P.SubTotal * (CS1.Valor / 100))
                    when (TPD.Codigo in ('PEDNORMAL', 'PEDMESA')) then (P.SubTotal * (CS1.Valor / 100))
                end
            ) AS ComisionVentaNeto,
            (
                case
                    when (
                        if(
                            isnull(P.IdPedidoBase),
                            'NO',
                            'SI'
                        ) = 'SI'
                    ) then 0
                    when (TPD.Codigo = 'PEDAGILFAVOR') then 0
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PPE.Nombre = 'Activo')
                    ) then (
                        (P.SubTotal * (CS1.Valor / 100)) * (PP.MedioPagoIVA / 100)
                    )
                    when (TPD.Codigo in ('PEDNORMAL', 'PEDMESA')) then (
                        (P.SubTotal * (CS1.Valor / 100)) * (PP.MedioPagoIVA / 100)
                    )
                end
            ) AS ComisionVentaIva,
            (
                case
                    when (
                        if(
                            isnull(P.IdPedidoBase),
                            'NO',
                            'SI'
                        ) = 'SI'
                    ) then 0
                    when (TPD.Codigo = 'PEDAGILFAVOR') then 0
                    when (
                        (TPD.Codigo = 'PEDESPECIAL')
                        and (PPE.Nombre = 'Activo')
                    ) then (
                        (P.SubTotal * (CS1.Valor / 100)) + (
                            (P.SubTotal * (CS1.Valor / 100)) * (PP.MedioPagoIVA / 100)
                        )
                    )
                    when (TPD.Codigo in ('PEDNORMAL', 'PEDMESA')) then (
                        (P.SubTotal * (CS1.Valor / 100)) + (
                            (P.SubTotal * (CS1.Valor / 100)) * (PP.MedioPagoIVA / 100)
                        )
                    )
                end
            ) AS ComisionVentaBruto,
            ifnull(
                (
                    case
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                        ) then CS2.Valor
                        when (TPD.Codigo = 'PEDAGILFAVOR') then 0
                    end
                ),
                0
            ) AS ComisionDeliveryPorcentaje,
            ifnull(
                (
                    case
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                        ) then (P.SubTotal * (CS2.Valor / 100))
                        when (TPD.Codigo = 'PEDAGILFAVOR') then FIJOS.FlashCostoDelivery
                    end
                ),
                0
            ) AS ComisionDeliveryNeto,
            ifnull(
                (
                    case
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                            and (if(isnull(P.IdPedidoBase), 'NO', 'SI') = 'SI')
                        ) then (
                            (P.SubTotal * (CS2.Valor / 100)) * (PPP.MedioPagoIVA / 100)
                        )
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                        ) then (
                            (P.SubTotal * (CS2.Valor / 100)) * (PP.MedioPagoIVA / 100)
                        )
                        when (TPD.Codigo = 'PEDAGILFAVOR') then (FIJOS.FlashCostoDelivery * (N.IVA / 100))
                    end
                ),
                0
            ) AS ComisionDeliveryIva,
            ifnull(
                (
                    case
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                            and (if(isnull(P.IdPedidoBase), 'NO', 'SI') = 'SI')
                        ) then (
                            (P.SubTotal * (CS2.Valor / 100)) + (
                                (P.SubTotal * (CS2.Valor / 100)) * (PPP.MedioPagoIVA / 100)
                            )
                        )
                        when (
                            (
                                TPD.Codigo in ('PEDNORMAL', 'PEDESPECIAL', 'PEDMESA')
                            )
                            and (PTE.Tipo = 'Delivery')
                        ) then (
                            (P.SubTotal * (CS2.Valor / 100)) + (
                                (P.SubTotal * (CS2.Valor / 100)) * (PP.MedioPagoIVA / 100)
                            )
                        )
                        when (TPD.Codigo = 'PEDAGILFAVOR') then (
                            FIJOS.FlashCostoDelivery + (FIJOS.FlashCostoDelivery * (N.IVA / 100))
                        )
                    end
                ),
                0
            ) AS ComisionDeliveryBruto,
            if(
                (SD.IdSucursalDeposito is not null),
                cast(SD.FechaDeposito as date),
                (
                    (
                        (
                            SCT.InicioPeriodoFacturacion + interval (
                                SCT.CantidadDiasPeriodos * truncate(
                                    (
                                        (
                                            to_days(PE.FechaEstimadaEntrega) - to_days(SCT.InicioPeriodoFacturacion)
                                        ) / SCT.CantidadDiasPeriodos
                                    ),
                                    0
                                )
                            ) day
                        ) + interval (SCT.CantidadDiasPeriodos - 1) day
                    ) + interval SCT.DiaPagoPosteriorAlCierre day
                )
            ) AS DiaDepositoComercio,
            if(
                (SD.IdSucursalDeposito is not null),
                (
                    (
                        to_days(SD.PeriodoHasta) - to_days(SD.PeriodoDesde)
                    ) + 1
                ),
                SCT.CantidadDiasPeriodos
            ) AS CantidadDiasPeriodo,
            SD.IdSucursalDeposito AS IdSucursalDeposito,
            if(
                (SD.IdSucursalDeposito is not null),
                SD.PeriodoDesde,
                (
                    SCT.InicioPeriodoFacturacion + interval (
                        SCT.CantidadDiasPeriodos * truncate(
                            (
                                (
                                    to_days(PE.FechaEstimadaEntrega) - to_days(SCT.InicioPeriodoFacturacion)
                                ) / SCT.CantidadDiasPeriodos
                            ),
                            0
                        )
                    ) day
                )
            ) AS PeriodoFacturacionInicioComercio,
            if(
                (SD.IdSucursalDeposito is not null),
                SD.PeriodoHasta,
                (
                    (
                        SCT.InicioPeriodoFacturacion + interval (
                            SCT.CantidadDiasPeriodos * truncate(
                                (
                                    (
                                        to_days(PE.FechaEstimadaEntrega) - to_days(SCT.InicioPeriodoFacturacion)
                                    ) / SCT.CantidadDiasPeriodos
                                ),
                                0
                            )
                        ) day
                    ) + interval (SCT.CantidadDiasPeriodos - 1) day
                )
            ) AS PeriodoFacturacionFinComercio,
            (
                (
                    (
                        FIJOS.InicioFacturacionRider + interval (
                            FIJOS.CantidadDiasPeriodoRider * truncate(
                                (
                                    (
                                        to_days(PE.FechaEstimadaEntrega) - to_days(FIJOS.InicioFacturacionRider)
                                    ) / FIJOS.CantidadDiasPeriodoRider
                                ),
                                0
                            )
                        ) day
                    ) + interval (FIJOS.CantidadDiasPeriodoRider - 1) day
                ) + interval FIJOS.DiaSemanaDepositoRider day
            ) AS DiaDepositoRider,
            (
                FIJOS.InicioFacturacionRider + interval (
                    FIJOS.CantidadDiasPeriodoRider * truncate(
                        (
                            (
                                to_days(PE.FechaEstimadaEntrega) - to_days(FIJOS.InicioFacturacionRider)
                            ) / FIJOS.CantidadDiasPeriodoRider
                        ),
                        0
                    )
                ) day
            ) AS PeriodoFacturacionInicioRider,
            (
                (
                    FIJOS.InicioFacturacionRider + interval (
                        FIJOS.CantidadDiasPeriodoRider * truncate(
                            (
                                (
                                    to_days(PE.FechaEstimadaEntrega) - to_days(FIJOS.InicioFacturacionRider)
                                ) / FIJOS.CantidadDiasPeriodoRider
                            ),
                            0
                        )
                    ) day
                ) + interval (FIJOS.CantidadDiasPeriodoRider - 1) day
            ) AS PeriodoFacturacionFinRider,
            R.Id_Rider AS IdRider,
            R.Id_Publico AS IdPublicoRider,
            R.DNI AS DniRider,
            R.Nombre AS NombreRider,
            R.Apellido AS ApellidoRider,
            R.Mail AS CorreoRider,
            RFV.TarifaBase AS TarifaBaseRider,
            RFV.PrecioDistanciaExtra AS IngresoExtraRider,
            RFV.IngresoNeto AS IngresoNetoRider,
            RFV.RetencionHonorarios AS RetencionHonorarioRider,
            RFV.IngresoBruto AS IngresoBrutoRider,
            RFV.IngresoViaje AS IngresoViajeRider,
            0 AS PropinaRider,
            RFV.IngresoTotalViaje AS TotalOfertaRider,
            RFV.IngresoTotal AS DepositoRider,
            P.IdCliente AS IdCliente,
            TP.tipo_pago AS TipoPago,
            RTV.Id_TipoVehiculo AS Id_TipoVehiculo,
            RTV.Descripcion AS Descripcion,
            ifnull(
                (
                    (P.CuotaServicioAgil * 100) / (100 + ifnull(PP.MedioPagoIVA, N.IVA))
                ),
                0
            ) AS CuotaServicioAgilNeto,
            ifnull(
                (
                    (
                        P.CuotaServicioAgil * ifnull(PP.MedioPagoIVA, N.IVA)
                    ) / (100 + ifnull(PP.MedioPagoIVA, N.IVA))
                ),
                0
            ) AS CuotaServicioAgilIVA,
            ifnull(P.CuotaServicioAgil, 0) AS CuotaServicioAgilBruto,
            ifnull(P.CuotaPasarelaNeto, 0) AS CuotaPasarelaNeto,
            ifnull(P.CuotaPasarelaIVA, 0) AS CuotaPasarelaIVA,
            ifnull(P.CuotaPasarelaBruto, 0) AS CuotaPasarelaBruto,
            ifnull(MPV.FeePay, 0) AS FeePay
        from (
                (
                    (
                        (
                            (
                                (
                                    (
                                        (
                                            (
                                                (
                                                    (
                                                        (
                                                            (
                                                                (
                                                                    (
                                                                        (
                                                                            (
                                                                                (
                                                                                    (
                                                                                        (
                                                                                            (
                                                                                                (
                                                                                                    (
                                                                                                        (
                                                                                                            (
                                                                                                                (
                                                                                                                    delivery.pedidos P
                                                                                                                    join delivery.sucursal S on ((P.IdSucursal = S.IdSucursal))
                                                                                                                )
                                                                                                                join delivery.comercio C on ((S.IdComercio = C.IdComercio))
                                                                                                            )
                                                                                                            left join delivery.pedido_estado PES on ((PES.Id = P.IdEstado))
                                                                                                        )
                                                                                                        left join delivery.pedido_entrega PE on ((PE.IdPedido = P.IdPedido))
                                                                                                    )
                                                                                                    left join delivery.pedido_entrega_tipo_entrega PTE on ((PTE.Id = P.TipoEntrega))
                                                                                                )
                                                                                                left join delivery.pedido_pago PP on ((PP.IdPedido = P.IdPedido))
                                                                                            )
                                                                                            left join delivery.tipo_pago TP on ((TP.id_tipo_pago = PP.IdTipoPago))
                                                                                        )
                                                                                        left join delivery.pedido_pago_respuesta PPR on ((PPR.idPago = PP.IdPago))
                                                                                    )
                                                                                    left join delivery.pedido_pago_estado PPE on ((PPE.Id = PP.IdEstado))
                                                                                )
                                                                                left join delivery.tipo_pedido TPD on ((TPD.IdTipoPedido = P.IdTipoPedido))
                                                                            )
                                                                            left join delivery.nacionalidad N on ((N.Id = C.IdNacionalidad))
                                                                        )
                                                                        left join delivery.pedido_cupon PC on ((PC.IdPedido = P.IdPedido))
                                                                    )
                                                                    left join delivery.sucursal_cupon SC on ((SC.Id = PC.IdCupon))
                                                                )
                                                                left join delivery.medio_pago MP on ((MP.id = PP.IdMedioPago))
                                                            )
                                                            left join delivery.comision_sucursal CS1 on (
                                                                (
                                                                    (CS1.IdSucursal = S.IdSucursal)
                                                                    and (CS1.IdTipoComision = 1)
                                                                )
                                                            )
                                                        )
                                                        left join delivery.tipo_comision_sucursal TCS1 on ((TCS1.Id = CS1.IdTipoComision))
                                                    )
                                                    left join delivery.comision_sucursal CS2 on (
                                                        (
                                                            (CS2.IdSucursal = S.IdSucursal)
                                                            and (CS2.IdTipoComision = 2)
                                                        )
                                                    )
                                                )
                                                left join delivery.tipo_comision_sucursal TCS2 on ((TCS2.Id = CS2.IdTipoComision))
                                            )
                                            left join delivery.sucursal_contrato SCT on ((SCT.IdSucursal = S.IdSucursal))
                                        )
                                        left join delivery.sucursal_deposito SD on (
                                            (
                                                (SD.IdSucursal = S.IdSucursal)
                                                and (SD.IdSucursalDeposito = P.IdSucursalDeposito)
                                                and (SD.FechaDeposito is not null)
                                            )
                                        )
                                    )
                                    left join delivery.pedido_pago PPP on ((PPP.IdPedido = P.IdPedidoBase))
                                )
                                left join delivery.rider R on ((R.Id_Rider = PE.IdRider))
                            )
                            left join delivery.rider_facturacion_viajes RFV on (
                                (
                                    (RFV.IdPedido = P.IdPedido)
                                    and (RFV.IdRider = R.Id_Rider)
                                )
                            )
                        )
                        left join delivery.rider_tipo_vehiculo RTV on (
                            (RTV.Id_TipoVehiculo = P.IdTipoVehiculoRequerido)
                        )
                    )
                    left join delivery.medio_pago_validacion MPV on ((MPV.IdPedido = P.IdPedido))
                )
                join (
                    select '2021-02-01' AS InicioFacturacionRider,
                        7 AS CantidadDiasPeriodoRider,
                        3 AS DiaSemanaDepositoRider,
                        1000 AS FlashCostoDelivery
                ) FIJOS
            )
        where (
                (C.CodigoCentroNegocio = 'AGIL_SOFT')
                and (C.IdComercio <> 8)
                and (
                    date_format(PE.FechaEstimadaEntrega, '%Y-%m-%d') >= date_format(
                        (
                            now() - interval (((weekday(now()) + 7) % 7) + 35) day
                        ),
                        '%Y-%m-%d'
                    )
                )
            )
        order by P.IdPedido desc
    ) X
"""

In [10]:
df_base_financiera = call_query()

TypeError: call_query() missing 1 required positional argument: 'sql_query'